In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import BallTree
from sklearn.base import BaseEstimator
from sklearn.pipeline import make_pipeline

In [8]:
"../src/data/"+data

'../src/dataseason4_script.csv'

In [10]:
lines = pd.DataFrame()
for data in os.listdir("../src/data"):
    if data !='links.csv':
        temp = pd.read_csv("../src/data/"+data)
        temp.drop("Unnamed: 0", axis=1, inplace=True)
        lines = lines.append(temp)
        print(lines.shape)
    else:
        pass
        

(5223, 4)
(10310, 4)
(16433, 4)
(22629, 4)
(28298, 4)
(34440, 4)
(35739, 4)
(42154, 4)


In [11]:
# lines = pd.read_csv("../data/season1_script.csv")
# lines.drop("Unnamed: 0", axis=1, inplace=True)

In [12]:
lines['context'] = lines['script'].shift(1)


In [20]:
lines.head()

,season,episode,friend,script,context
0,season4,401jelly,Joey,"(lying on a beach towel, recapping what happe...",NaN
1,season4,401jelly,Phoebe Sr.,(to Phoebe) Im your mother.,"(lying on a beach towel, recapping what happe..."
2,season4,401jelly,Phoebe,Ehh?,(to Phoebe) Im your mother.
3,season4,401jelly,Joey,"(voice-over) Oh, and then Monica joked that s...",Ehh?
4,season4,401jelly,Chandler,(to Monica) (in a funny voice) Hi there. (Mon...,"(voice-over) Oh, and then Monica joked that s..."


In [21]:
chandler = lines[lines['friend']=='Chandler']
chandler = chandler[~chandler['context'].isnull()]


chandler.head(4)

,season,episode,friend,script,context
4,season4,401jelly,Chandler,(to Monica) (in a funny voice) Hi there. (Mon...,"(voice-over) Oh, and then Monica joked that s..."
47,season4,401jelly,Chandler,"(jumping up) Oh, Ill go out and get you some.",(closing the fridge in disgust) Shoot! Were ...
49,season4,401jelly,Chandler,Nope! Because Im not your boyfriend. (to Pho...,Really?!
51,season4,401jelly,Chandler,I found a dried up seashores.,"Well, umm, my Moms friend, Phoebe, is actual..."


In [22]:
chandler.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5848 entries, 4 to 6397
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   season   5848 non-null   object
 1   episode  5848 non-null   object
 2   friend   5848 non-null   object
 3   script   5848 non-null   object
 4   context  5848 non-null   object
dtypes: object(5)
memory usage: 274.1+ KB


In [23]:
vector = TfidfVectorizer()
matrix = vector.fit_transform(chandler.context)
print(matrix.shape)

(5848, 5069)


In [26]:
svd = TruncatedSVD(n_components=2000)
smal_matr = svd.fit_transform(matrix)
print(smal_matr.shape)
print(svd.explained_variance_ratio_.sum())

(5848, 2000)
0.9555890481816578


In [27]:
def softmax(x):
    proba = np.exp(-x)
    return proba/sum(proba)


class NeighbourCample(BaseEstimator):
    
    def __init__(self, k=5, temperature=1.0):
        self.k = k
        self.temperature = temperature
        
    def fit(self, X, y):
        self.tree_ = BallTree(X)
        self.y_ = np.array(y)
        
    def predict(self, X, random_state=None):
        distances, indices = self.tree_.query(X, return_distance=True, k=self.k)
        result = []
        for distance, index in zip(distances, indices):
            result.append(np.random.choice(index, p = softmax(distance * self.temperature)))
        return self.y_[result]        

In [28]:
ns = NeighbourCample()
ns.fit(smal_matr, chandler.script)
pipe = make_pipeline(vector, svd, ns)

In [37]:
print(pipe.predict(['coffee']))

[' Well, I am drinking lots of cups of coffee because I\x92m exhausted!\nBecause Joey started snoring!']


In [213]:
token = "1783843952:AAGkKSVzqjo5HlwmDDABI4t3AEcPqw54cRw"